# Import libraries & settings

In [1]:
import os

import pandas as pd

from bs4 import BeautifulSoup

import shutil

from tqdm import tqdm

import numpy as np

In [2]:
# Settings
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 200)

In [3]:
# Path to root folder
dir_path = 'C:/Users/Adam/Documents/Out of the Park Developments/OOTP Baseball 24/saved_games/'

# Destination folder for processed game logs
destination_folder_pitchers = 'C:/Users/Adam/Desktop/OOTP/Pitchers_historical_logs'
destination_folder_batters = 'C:/Users/Adam/Desktop/OOTP/Batters_historical_logs'

# Destination folder for final data sets
destination_folder = 'C:/Users/Adam/Desktop/OOTP'

# Import and process logs

In [4]:
# Import HTML files
html_files = []

for root, dirs, files in os.walk(dir_path):
    if 'html' in dirs:
        temp_dir = os.path.join(root, 'html')
        if 'temp' in os.listdir(temp_dir):
            html_dir = os.path.join(temp_dir, 'temp')
            for file in os.listdir(html_dir):
                if file.endswith('.html'):
                    html_files.append(os.path.join(html_dir, file))


In [5]:
# Load the parquet "DBs" into a Pandas DataFrame
# https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d
# feather was impossible to set up
df_pitchers = pd.read_parquet(r'C:\Users\Adam\Desktop\OOTP\pitchers_db.parquet')
df_batters = pd.read_parquet(r'C:\Users\Adam\Desktop\OOTP\batters_db.parquet')

In [6]:
# ETL for new logs

for file in tqdm(html_files):
    # Reading the HTML file into a pandas DataFrame
    df = pd.read_html(file, flavor = 'bs4', skiprows=[0])[0]
    
    # delete the first row
    df = df.drop(0)
    # set the header and drop the row
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    
    # Check for duplicate column names
    duplicate_cols = df.columns[df.columns.duplicated()]
    # print(duplicate_cols) 

    # Get a boolean mask indicating which column names are duplicates
    dupe_mask = df.columns.duplicated()

    # Add a suffix to duplicate column names
    df.columns = df.columns.where(~dupe_mask, df.columns + '_other')
    
    # Concatenate the DataFrames for pitchers and batters independently
    if df.columns[6] == "STU":
        df_pitchers = pd.concat([df_pitchers, df])
        # Move
        #shutil.move(file, destination_folder_pitchers)
    else:
        # Drop first column for batters
        df = df.drop(df.columns[0], axis=1)
        # Concat
        df_batters = pd.concat([df_batters, df])
        # Move
        #shutil.move(file, destination_folder_batters)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [03:28<00:00, 34.76s/it]


In [7]:
# Store unaggregated data between sessions in parquet file
df_pitchers.to_parquet(r'C:\Users\Adam\Desktop\OOTP\pitchers_db.parquet')
df_batters.to_parquet(r'C:\Users\Adam\Desktop\OOTP\batters_db.parquet')

# Prepare Dataframes

## Data Types

### Pitchers

In [8]:
pitchers_data_types = {
    "POS": str,
    "Name": str,
    "ORG": str,
    "B": str,
    "T": str,
    "OVR": int,
    "STU": int,
    "MOV": int,
    "CON": int,
    "PBABIP": int,
    "HRR": int,
    "STU vL": int,
    "MOV vL": int,
    "CON vL": int,
    "PBABIP vL": int,
    "HRR vL": int,
    "STU vR": int,
    "MOV vR": int,
    "CON vR": int,
    "PBABIP vR": int,
    "HRR vR": int,
    "FB": int,
    "CH": int,
    "CB": int,
    "SL": int,
    "SI": int,
    "SP": int,
    "CT": int,
    "FO": int,
    "CC": int,
    "SC": int,
    "KC": int,
    "KN": int,
    "PIT": int,
    "G/F": str,
    "VELO": str,
    "Slot": str,
    "PT": str,
    "STM": int,
    "HLD": int,
    "IF RNG": int,
    "IF ERR": int,
    "IF ARM": int,
    "P": int,
    "G": int,
    "GS": int,
    "W": int,
    "L": int,
    "WIN%": float,
    "SVO": int,
    "SV": int,
    "SV%": float,
    "BS": int,
    "BS%": float,
    "HLD_other": int,
    "SD": int,
    "MD": int,
    "IP": float,
    "BF": int,
    "AB": int,
    "HA": int,
    "1B": int,
    "2B": int,
    "3B": int,
    "HR": int,
    "TB": int,
    "R": int,
    "ER": int,
    "BB": int,
    "IBB": int,
    "K": int,
    "HP": int,
    "ERA": float,
    "AVG": float,
    "OBP": float,
    "SLG": float,
    "OPS": float,
    "BABIP": float,
    "WHIP": float,
    "RA/9": float,
    "HR/9": float,
    "H/9": float,
    "BB/9": float,
    "K/9": float,
    "K/BB": float,
    "K%": float,
    "BB%": float,
    "K%-BB%": float,
    "SH": int,
    "SF": int,
    "WP": int,
    "BK": int,
    "CI": int,
    "DP": int,
    "RA": int,
    "GF": int,
    "IR": int,
    "IRS": int,
    "IRS%": float,
    "LOB%": float,
    "pLi": float,
    "GF%": float,
    "QS": int,
    "QS%": float,
    "CG": int,
    "CG%": float,
    "SHO": int,
    "PPG": int,
    "RS": int,
    "RSG": float,
    "PI": int,
    "GB": int,
    "FB_other": int,
    "GO%": float,
    "SB": int,
    "CS": int,
    "ERA+": int,
    "FIP": float,
    "FIP-": float,
    "WPA": float,
    "WAR": float,
    "rWAR": float,
    "SIERA": float,
    "TC": int,
    "A": int,
    "PO": int,
    "E": int,
    "DP_other": int,
    "TP": int,
    "PCT": float,
    "RNG": float,
    "ZR": float,
    "EFF": float
}

In [9]:
for col, dtype in pitchers_data_types.items():
    if df_pitchers[col].dtype != dtype and dtype != 'str':
        df_pitchers[col] = df_pitchers[col].replace('-', 0)
        #df_pitchers[col] = pd.to_numeric(df_pitchers[col], errors='coerce').astype(dtype)
        df_pitchers[col] = df_pitchers[col].astype(dtype, errors = 'ignore')
        

In [10]:
df_pitchers = df_pitchers.convert_dtypes().reset_index(drop=True)
df_pitchers['VELO'] = df_pitchers['VELO'].str.split('-').str[0]

### Batters

In [11]:
batters_data_types = {'POS': str, 'Name': str, 'ORG': str, 'Lev': str, 'HT': str, 'B': str, 'T': str, 'OVR': int, 
                'BABIP': int, 'CON': int, 'GAP': int, 'POW': int, 'EYE': int, 'K\'s': int, 'BABIP vL': int, 
                'CON vL': int, 'GAP vL': int, 'POW vL': int, 'EYE vL': int, 'K vL': int, 'BABIP vR': int, 
                'CON vR': int, 'GAP vR': int, 'POW vR': int, 'EYE vR': int, 'K vR': int, 'BUN': int, 'BFH': int, 
                'BBT': str, 'GBT': str, 'FBT': str, 'C ABI': int, 'C ARM': int, 'IF RNG': int, 'IF ERR': int, 
                'IF ARM': int, 'TDP': int, 'OF RNG': int, 'OF ERR': int, 'OF ARM': int, 'DEF': int, 'P': int, 
                'C': int, '1B': int, '2B': int, '3B': int, 'SS': int, 'LF': int, 'CF': int, 'RF': int, 'SPE': int, 
                'STE': int, 'RUN': int, 'G': int, 'GS': int, 'PA': int, 'AB': int, 'H': int, '1B_other': int, 
                '2B_other': int, '3B_other': int, 'HR': int, 'RBI': int, 'R': int, 'BB': int, 'BB%': float, 'IBB': int, 
                'HP': int, 'SH': int, 'SF': int, 'CI': int, 'SO': int, 'SO%': float, 'GIDP': int, 'EBH': int, 'TB': int, 
                'AVG': float, 'OBP': float, 'SLG': float, 'RC': float, 'RC/27': float, 'ISO': float, 'wOBA': float, 'OPS': float,  
                'OPS+': int, 'BABIP_other': float, 'WPA': float, 'wRC': int, 'wRC+': int, 'wRAA': float, 'WAR': float, 
                'PI/PA': float, 'SB': int, 'CS': int, 'SB%': float, 'BatR': float, 'wSB': float, 'UBR': float, 'BsR': float, 
                'G_other': int, 'GS_other': int, 'TC': int, 'A': int, 'PO': int, 'E': int, 'DP': int, 'TP': int, 
                'PCT': float, 'RNG': float, 'ZR': float, 'EFF': float, 'SBA': int, 'RTO': int, 'RTO%': float, 'IP': float, 
                'PB': int, 'CER': int, 'CERA': float, 'BIZ-R%': float, 'BIZ-R': int, 'BIZ-Rm': int, 'BIZ-L%': float, 
                'BIZ-L': int, 'BIZ-Lm': int, 'BIZ-E%': float, 'BIZ-E': int, 'BIZ-Em': int, 'BIZ-U%': float, 
                'BIZ-U': int, 'BIZ-Um': int, 'BIZ-Z%': float, 'BIZ-Z': int, 'BIZ-Zm': int, 'BIZ-I': int, 'FRM': float, 
                'ARM': float, 'BF%': float}


In [12]:
for col, dtype in batters_data_types.items():
    if df_batters[col].dtype != dtype and dtype != 'str':
        df_batters[col] = df_batters[col].replace('-', 0)
        #df_batters[col] = pd.to_numeric(df_batters[col], errors='coerce').astype(dtype)
        df_batters[col] = df_batters[col].astype(dtype, errors = 'ignore')

In [13]:
df_batters = df_batters.convert_dtypes().reset_index(drop=True)

## Aggregations

In [21]:
df_pitchers
df_batters

,POS,Name,ORG,Lev,HT,B,T,OVR,BABIP,CON,GAP,POW,EYE,K's,BABIP vL,CON vL,GAP vL,POW vL,EYE vL,K vL,BABIP vR,CON vR,GAP vR,POW vR,EYE vR,K vR,BUN,BFH,BBT,GBT,FBT,C ABI,C ARM,IF RNG,IF ERR,IF ARM,TDP,OF RNG,OF ERR,OF ARM,DEF,P,C,1B,2B,3B,SS,LF,CF,RF,SPE,STE,RUN,G,GS,PA,AB,H,1B_other,2B_other,3B_other,HR,RBI,R,BB,BB%,IBB,HP,SH,SF,CI,SO,SO%,GIDP,EBH,TB,AVG,OBP,SLG,RC,RC/27,ISO,wOBA,OPS,OPS+,BABIP_other,WPA,wRC,wRC+,wRAA,WAR,PI/PA,SB,CS,SB%,BatR,wSB,UBR,BsR,G_other,GS_other,TC,A,PO,E,DP,TP,PCT,RNG,ZR,EFF,SBA,RTO,RTO%,IP,PB,CER,CERA,BIZ-R%,BIZ-R,BIZ-Rm,BIZ-L%,BIZ-L,BIZ-Lm,BIZ-E%,BIZ-E,BIZ-Em,BIZ-U%,BIZ-U,BIZ-Um,BIZ-Z%,BIZ-Z,BIZ-Zm,BIZ-I,FRM,ARM,BF%
0,RP,Jimmy Ryan,Yakima Apple Sox,TRN,"5' 9""",Right,Left,59,84,84,83,67,63,52,85,86,84,68,64,52,84,84,82,66,62,52,6,2,Line Drive,Normal,Spray,0,0,28,23,25,24,73,56,80,60,60,0,0,0,0,0,0,59,0,88,91,89,4,4,16,14,4,3,0,0,1,1,3,2,12.5,0,0,0,0,0,4,25.0,0,1,7,0.286,0.375,0.5,2.7,6.6,0.214,0.375,0.875,157,0.333,-0.02,2,161,1.0,0.1,5.25,2,1,66.7,1.0,0.1,0.0,0.1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0,0.0,0.0,100%
1,C,Luis Campusano,Camden New Jerseys,TRN,"5' 10""",Right,Right,68,56,62,51,62,54,56,57,63,52,63,55,57,56,62,51,62,54,56,35,17,Normal,Pull,Pull,60,76,34,33,67,6,10,27,44,62,0,62,46,0,0,0,0,0,0,23,25,36,4,4,18,15,1,1,0,0,0,0,1,3,16.7,0,0,0,0,0,4,22.2,0,0,1,0.067,0.222,0.067,0.4,0.8,0.0,0.157,0.289,-7,0.091,-0.26,0,-5,-1.9,-0.1,4.5,0,0,0.0,-1.9,0.0,0.4,0.4,4,4,29,2,27,0,0,0,1.0,7.68,-0.0,1.0,2,0,0.0,34.0,0,14,3.71,0.0%,0,0,0.0%,0,0,100.0%,1,1,0.0%,0,0,0.0%,0,0,0,0.1,0.0,100%
2,C,Luis Campusano,Skep West,TRN,"5' 10""",Right,Right,68,56,62,51,62,54,56,57,63,52,63,55,57,56,62,51,62,54,56,35,17,Normal,Pull,Pull,60,76,34,33,67,6,10,27,44,62,0,62,46,0,0,0,0,0,0,23,25,36,3,3,11,10,1,1,0,0,0,0,1,1,9.1,0,0,0,0,0,0,0.0,0,0,1,0.1,0.182,0.1,0.2,0.7,0.0,0.137,0.282,-12,0.1,-0.12,0,-20,-1.3,-0.2,2.73,0,0,0.0,-1.3,0.0,-0.3,-0.3,3,3,22,2,20,0,0,0,1.0,7.62,0.1,0.0,2,1,50.0,26.0,0,12,4.15,0.0%,0,0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0,-0.8,0.0,100%
3,C,Luis Campusano,So Cal Spicy Srirachas,TRN,"5' 10""",Right,Right,68,56,62,51,62,54,56,57,63,52,63,55,57,56,62,51,62,54,56,35,17,Normal,Pull,Pull,60,76,34,33,67,6,10,27,44,62,0,62,46,0,0,0,0,0,0,23,25,36,4,4,15,15,3,3,0,0,0,1,0,0,0.0,0,0,0,0,0,2,13.3,0,0,3,0.2,0.2,0.2,0.6,1.3,0.0,0.171,0.4,20,0.231,-0.26,0,5,-1.4,-0.0,3.47,0,0,0.0,-1.4,0.0,0.1,0.1,4,4,28,2,26,0,0,0,1.0,7.41,0.2,1.0,2,1,50.0,34.0,0,9,2.38,0.0%,0,0,0.0%,0,0,100.0%,1,1,0.0%,0,0,0.0%,0,0,0,0.0,0.0,100%
4,C,Luis Campusano,Skep North,TRN,"5' 10""",Right,Right,68,56,62,51,62,54,56,57,63,52,63,55,57,56,62,51,62,54,56,35,17,Normal,Pull,Pull,60,76,34,33,67,6,10,27,44,62,0,62,46,0,0,0,0,0,0,23,25,36,4,4,14,13,4,4,0,0,0,1,1,1,7.1,0,0,0,0,0,2,14.3,0,0,4,0.308,0.357,0.308,1.5,4.6,0.0,0.292,0.665,101,0.364,0.12,1,97,-0.0,0.2,2.71,0,0,0.0,-0.0,0.0,0.0,0.0,4,4,21,1,20,0,0,0,1.0,5.56,0.1,0.0,2,1,50.0,34.0,0,14,3.71,0.0%,0,0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0,0.5,0.0,100%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1127,RF,Reggie Sanders,Fogtown Stars,TRN,"6' 1""",Right,Right,68,62,66,65,74,76,45,62,67,67,75,81,46,62,65,64,73,75,45,5,6,Flyball,Pull,Normal,0,0,1,8,4,6,66,58,41,57,0,0,0,0,0,0,72,42,57,88,82,65,3,<NA>,4,4,1,1,0,0,0,2,0,0,0.0,0,0,0,0,0,0,0.0,0,0,1,0.25,0.25,0.25,0.2,2.2,0.0,0.225,0.5,42,0.25,0.12,0,33,-0.3,-0.0,3.5,0,0,0.0,-0.3,0.0,0.0,0.0,2,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0.0,4.0,0,0,0.0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0.0%,0,0,0,0.0,0.0

In [25]:
print(df_batters.iloc[:, 53:])

      G    GS  PA  AB  H  1B_other  2B_other  3B_other  HR  RBI  R  BB   BB%  \
0     4     4  16  14  4         3         0         0   1    1  3   2  12.5   
1     4     4  18  15  1         1         0         0   0    0  1   3  16.7   
2     3     3  11  10  1         1         0         0   0    0  1   1   9.1   
3     4     4  15  15  3         3         0         0   0    1  0   0   0.0   
4     4     4  14  13  4         4         0         0   0    1  1   1   7.1   
...  ..   ...  ..  .. ..       ...       ...       ...  ..  ... ..  ..   ...   
1127  3  <NA>   4   4  1         1         0         0   0    2  0   0   0.0   
1128  2  <NA>   4   4  0         0         0         0   0    0  0   0   0.0   
1129  2     2   9   9  2         1         1         0   0    1  0   0   0.0   
1130  6     5  19  16  5         5         0         0   0    2  1   3  15.8   
1131  2     2  12   9  1         0         0         0   1    2  1   3  25.0   

      IBB  HP  SH  SF  CI  SO   SO%  GI

In [20]:
print(df_pitchers.iloc[:, 44:])


     G  GS  W  L  WIN%  SVO  SV  SV%  BS  BS%  HLD_other  SD  MD    IP  BF  \
0    1   1  0  0   0.0    0   0  0.0   0  0.0          0   0   0   5.0  21   
1    2   2  2  0   1.0    0   0  0.0   0  0.0          0   0   0  12.2  50   
2    1   1  0  1   0.0    0   0  0.0   0  0.0          0   0   0   6.0  26   
3    2   2  1  1   0.5    0   0  0.0   0  0.0          0   0   0  13.1  48   
4    1   1  0  1   0.0    0   0  0.0   0  0.0          0   0   0   1.2   9   
..  ..  .. .. ..   ...  ...  ..  ...  ..  ...        ...  ..  ..   ...  ..   
911  2   0  0  0   0.0    0   0  0.0   0  0.0          1   1   0   2.0   9   
912  2   0  0  0   0.0    0   0  0.0   0  0.0          1   1   0   3.0   9   
913  1   0  0  0   0.0    0   0  0.0   0  0.0          0   1   0   4.0  15   
914  3   0  0  0   0.0    1   1  1.0   0  0.0          0   0   0   5.1  20   
915  1   0  0  1   0.0    1   0  0.0   1  1.0          0   0   1   0.1   4   

     AB  HA  1B  2B  3B  HR  TB  R  ER  BB  IBB   K  HP   ERA  

In [124]:
df_pitchers_copy = df_pitchers.copy()

In [126]:
df_pitchers_copy = df_pitchers_copy.groupby(list(df_pitchers.columns[:53]))

In [78]:
# too slow
wm = lambda x: np.average(x, weights=df_pitchers.loc[x.index, 'IP']) if df_pitchers.loc[x.index, 'IP'].sum() > 0 else np.nan
ip9 = lambda x: np.average(x * (9/df_pitchers.loc[x.index, 'IP']), weights=df_pitchers.loc[x.index, 'IP']) if df_pitchers.loc[x.index, 'IP'].sum() > 0 else np.nan
lambda_fncs = {col: ['sum', 'mean', wm, ip9] for col in df_pitchers.columns[53:]}

%%time
pitch_agg = df_pitchers_copy.agg(lambda_fncs)

In [132]:
def wm(df, col, weight_col):
    weighted_avg_col = (df[col] * df[weight_col]).sum() / df[weight_col].sum()
    return weighted_avg_col

In [127]:
df_pitchers_copy.apply()

KeyError: "Int64Index([292, 750], dtype='int64') not in index"

In [84]:
def weighted_average(group):
   weights = group['Volume']
   height = group['Height']   
   return np.average(height,weights=weights)

df.groupby(['STAND_ID','Species']).apply(func = weighted_average)


CPU times: total: 438 ms
Wall time: 408 ms


In [ ]:
# df_pitchers_copy.groupby(list(df_pitchers.columns[:53])).apply(lambda x: (x['Height'] * x['Volume'].div(x['Volume'].sum())).sum()).unstack()


In [ ]:
weighted_avg = lambda x: (x['salary'] * x['weight']).sum() / x['weight'].sum()

In [133]:

# Aggregate first 53 columns
aggregated = df_pitchers.iloc[:, :53].agg(['sum', 'mean'])

# Calculate weighted average for remaining 10 columns
weighted_avg = df_pitchers.iloc[:, 53:].apply(lambda x: wm(df_pitchers, x.name, 'IP'))

C:\Users\Adam\AppData\Local\Temp\ipykernel_1032\3018357968.py:2: FutureWarning: ['POS', 'Name', 'ORG', 'B', 'T', 'G/F', 'Slot', 'PT'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  aggregated = df_pitchers.iloc[:, :53].agg(['sum', 'mean'])


In [134]:
weighted_avg

BS%          0.044955
HLD_other    0.108030
SD           0.328527
MD           0.143896
IP           5.707333
               ...   
TP           0.000000
PCT          0.377889
RNG          0.864274
ZR          -0.002440
EFF          0.246302
Length: 80, dtype: float64

In [62]:
pitch_agg

BS%  \
                                                                                                                                                                                                                                                                                                         sum   
POS Name           ORG                           B     T     OVR STU MOV CON PBABIP HRR STU vL MOV vL CON vL PBABIP vL HRR vL STU vR MOV vR CON vR PBABIP vR HRR vR FB CH CB SL SI SP CT FO CC SC KC KN PIT G/F   VELO Slot         PT      STM HLD IF RNG IF ERR IF ARM P  G GS W L WIN% SVO SV SV% BS        
CF  Jimmy Ryan     Goyang Cats                   Right Left  59  62  40  43  48     44  71     46     45     54        49     59     38     42     45        42     0  0  0  74 0  56 0  0  0  0  0  0  2   NEU   90   Normal (3/4) Normal  38  60  28     23     25     60 0 0  0 0 0.0  0   0  0.0 0   0.0   
CL  Ace Adams      Albuquerque Albatross         Right Right 69  61  95  53  83     90  60     93     53     81        87     62     97     54     85        92     63 33 52 64 0  0  0  0  0  0  0  0  4   FB    90   Over the Top Normal  21  86  23     18     20     46 1 0  0 0 0.0  1   1  1.0 0   0.0   
                   Staten Island Gunksters (F2P) Right Right 69  61  95  53  83     90  60     93     53     81        87     62     97     54     85        92     63 33 52 64 0  0  0  0  0  0  0  0  4   FB    90   Over the Top Normal  21  86  23     18     20     46 2 0  0 0 0.0  2   1  0.5 1   1.0   
    Alex Wood      Colorado Tommyknockers        Right Left  68  75  58  75  50     61  87     68     79     59        71     71     55     74     48        57     0  0  0  78 74 0  0  0  64 0  0  0  3   NEU   92   Normal (3/4) Normal  70  76  40     41     44     86 1 0  1 0 1.0  1   0  0.0 1   2.0   
                   Duskvol Vermin                Right Left  68  75  58  75  50     61  87     68     79     59        71     71     55     74     48        57     0  0  0  78 74 0  0  0  64 0  0  0  3   NEU   92   Normal (3/4) Normal  70  76  40     41     44     86 2 0  0 0 0.0  0   0  0.0 0   0.0   
...                                                                                                                                                                                                                                                                                                      ...   
SP  Tyler Anderson Hayden Maulers                Left  Left  67  54  62  80  66     61  56     65     81     67        64     54     62     80     66        60     60 53 32 0  57 0  53 0  0  0  0  0  5   FB    91   Normal (3/4) Finesse 68  68  32     28     51     76 1 1  0 0 0.0  0   0  0.0 0   0.0   
    Weldon Wyckoff Las Vegas Stacocakes          Right Right 54  76  116 28  69     114 70     107    27     64        105    80     124    28     73        121    75 0  0  81 0  72 0  0  0  0  0  0  3   NEU   90   Normal (3/4) Normal  91  55  27     22     22     56 2 0  0 0 0.0  1   1  1.0 0   0.0   
    Wilson Alvarez Narshe Returners              Left  Left  66  63  72  70  63     71  64     73     71     64        73     63     71     70     63        71     68 73 56 41 0  0  0  0  0  0  0  0  4   EX FB 91   Normal (3/4) Normal  41  64  28     33     28     70 1 1  0 0 0.0  0   0  0.0 0   0.0   
    Zane Smith     Duskvol Vermin                Left  Left  68  41  79  85  32     86  46     88     88     36        92     39     76     83     31        84     0  21 36 45 49 0  0  0  0  0  0  0  4   EX GB 89   Normal (3/4) GB'er   57  58  29     24     26     62 1 0  0 0 0.0  0   0  0.0 0   0.0   
                   Nagoya Grampus                Left  Left  68  41  79  85  32     86  46     88     88     36        92     39     76     83     31        84     0  21 36 45 49 0  0  0  0  0  0  0  4   EX GB 89   Normal (3/4) GB'er   57  58  29     24     26     62 1 1  1 0 1.0  0   0  0.0 0   0.0   

                                        

In [47]:
# Flatten column names
pitch_agg.columns = pitch_agg.columns.map('_'.join)

In [48]:
pitch_agg

BS%_sum  \
POS Name           ORG                           B     T     OVR STU MOV CON PBABIP HRR STU vL MOV vL CON vL PBABIP vL HRR vL STU vR MOV vR CON vR PBABIP vR HRR vR FB CH CB SL SI SP CT FO CC SC KC KN PIT G/F   VELO Slot         PT      STM HLD IF RNG IF ERR IF ARM P  G GS W L WIN% SVO SV SV% BS            
CF  Jimmy Ryan     Goyang Cats                   Right Left  59  62  40  43  48     44  71     46     45     54        49     59     38     42     45        42     0  0  0  74 0  56 0  0  0  0  0  0  2   NEU   90   Normal (3/4) Normal  38  60  28     23     25     60 0 0  0 0 0.0  0   0  0.0 0       0.0   
CL  Ace Adams      Albuquerque Albatross         Right Right 69  61  95  53  83     90  60     93     53     81        87     62     97     54     85        92     63 33 52 64 0  0  0  0  0  0  0  0  4   FB    90   Over the Top Normal  21  86  23     18     20     46 1 0  0 0 0.0  1   1  1.0 0       0.0   
                   Staten Island Gunksters (F2P) Right Right 69  61  95  53  83     90  60     93     53     81        87     62     97     54     85        92     63 33 52 64 0  0  0  0  0  0  0  0  4   FB    90   Over the Top Normal  21  86  23     18     20     46 2 0  0 0 0.0  2   1  0.5 1       1.0   
    Alex Wood      Colorado Tommyknockers        Right Left  68  75  58  75  50     61  87     68     79     59        71     71     55     74     48        57     0  0  0  78 74 0  0  0  64 0  0  0  3   NEU   92   Normal (3/4) Normal  70  76  40     41     44     86 1 0  1 0 1.0  1   0  0.0 1       2.0   
                   Duskvol Vermin                Right Left  68  75  58  75  50     61  87     68     79     59        71     71     55     74     48        57     0  0  0  78 74 0  0  0  64 0  0  0  3   NEU   92   Normal (3/4) Normal  70  76  40     41     44     86 2 0  0 0 0.0  0   0  0.0 0       0.0   
...                                                                                                                                                                                                                                                                                                          ...   
SP  Tyler Anderson Hayden Maulers                Left  Left  67  54  62  80  66     61  56     65     81     67        64     54     62     80     66        60     60 53 32 0  57 0  53 0  0  0  0  0  5   FB    91   Normal (3/4) Finesse 68  68  32     28     51     76 1 1  0 0 0.0  0   0  0.0 0       0.0   
    Weldon Wyckoff Las Vegas Stacocakes          Right Right 54  76  116 28  69     114 70     107    27     64        105    80     124    28     73        121    75 0  0  81 0  72 0  0  0  0  0  0  3   NEU   90   Normal (3/4) Normal  91  55  27     22     22     56 2 0  0 0 0.0  1   1  1.0 0       0.0   
    Wilson Alvarez Narshe Returners              Left  Left  66  63  72  70  63     71  64     73     71     64        73     63     71     70     63        71     68 73 56 41 0  0  0  0  0  0  0  0  4   EX FB 91   Normal (3/4) Normal  41  64  28     33     28     70 1 1  0 0 0.0  0   0  0.0 0       0.0   
    Zane Smith     Duskvol Vermin                Left  Left  68  41  79  85  32     86  46     88     88     36        92     39     76     83     31        84     0  21 36 45 49 0  0  0  0  0  0  0  4   EX GB 89   Normal (3/4) GB'er   57  58  29     24     26     62 1 0  0 0 0.0  0   0  0.0 0       0.0   
                   Nagoya Grampus                Left  Left  68  41  79  85  32     86  46     88     88     36        92     39     76     83     31        84     0  21 36 45 49 0  0  0  0  0  0  0  4   EX GB 89   Normal (3/4) GB'er   57  58  29     24     26     62 1 1  1 0 1.0  0   0  0.0 0       0.0   

                                                                                                                                                                                                                                                                                                    

In [50]:
pitch_agg[pitch_agg['BF_mean']!=pitch_agg['BF_<lambda_0>']]

BS%_sum  \
POS Name               ORG                    B      T     OVR STU MOV CON PBABIP HRR STU vL MOV vL CON vL PBABIP vL HRR vL STU vR MOV vR CON vR PBABIP vR HRR vR FB CH CB SL SI SP CT FO CC SC KC KN PIT G/F   VELO Slot         PT      STM HLD IF RNG IF ERR IF ARM P  G GS W L WIN% SVO SV SV% BS            
CF  Jimmy Ryan         Goyang Cats            Right  Left  59  62  40  43  48     44  71     46     45     54        49     59     38     42     45        42     0  0  0  74 0  56 0  0  0  0  0  0  2   NEU   90   Normal (3/4) Normal  38  60  28     23     25     60 0 0  0 0 0.0  0   0  0.0 0       0.0   
CL  Marshall Bridges   Chicago Yeezy's        Switch Left  65  91  96  30  75     92  89     94     30     74        91     91     96     30     75        93     83 0  88 0  0  0  0  0  0  0  0  0  2   NEU   93   Normal (3/4) Normal  17  74  28     24     26     61 1 0  0 0 0.0  0   0  0.0 0       0.0   
RP  Alex Cobb          Fogtown Stars          Right  Right 69  62  86  58  63     86  60     84     58     61        83     63     89     59     65        88     47 0  55 0  72 66 0  0  0  0  0  0  4   GB    91   Normal (3/4) GB'er   64  55  35     25     30     71 1 0  0 0 0.0  0   0  0.0 0       0.0   
    Andrew Vasquez     The Cat's Pajamas      Left   Left  56  74  76  45  53     78  75     77     45     55        80     74     76     45     53        78     0  0  87 0  50 0  0  0  0  0  0  0  2   EX GB 88   Normal (3/4) GB'er   19  56  30     16     50     62 1 0  0 0 0.0  0   0  0.0 0       0.0   
    Brady Singer       Knoxville Braves       Right  Right 61  73  55  63  52     58  65     49     61     46        51     80     60     65     56        64     0  62 0  78 72 0  0  0  0  0  0  0  3   NEU   94   Normal (3/4) Normal  61  42  41     31     57     81 1 0  1 0 1.0  0   0  0.0 0       0.0   
...                                                                                                                                                                                                                                                                                                        ...   
SP  Johnny Vander Meer Mighty Mallrats        Switch Left  52  87  113 20  73     110 83     108    20     70        106    88     114    20     74        111    85 74 90 0  0  0  0  0  0  0  0  0  3   NEU   93   Over the Top Normal  74  63  28     23     24     59 1 0  0 1 0.0  0   0  0.0 0       0.0   
    Jon Garland        Colorado Tommyknockers Right  Right 69  47  76  83  77     72  47     76     83     76        71     47     77     83     78        73     0  35 47 30 61 38 39 0  0  0  0  0  6   NEU   90   Over the Top Finesse 65  105 39     30     37     80 1 1  0 1 0.0  0   0  0.0 0       0.0   
    Lefty Leifield     FFS Snails             Left   Left  54  57  102 37  88     95  61     109    38     95        105    56     100    37     86        92     63 58 70 0  0  0  0  0  0  0  0  0  3   NEU   89   Sidearm      Normal  92  66  31     26     30     68 1 1  0 0 0.0  0   0  0.0 0       0.0   
                       Roswell Grays          Left   Left  54  57  102 37  88     95  61     109    38     95        105    56     100    37     86        92     63 58 70 0  0  0  0  0  0  0  0  0  3   NEU   89   Sidearm      Normal  92  66  31     26     30     68 1 0  0 0 0.0  0   0  0.0 0       0.0   
    Ron Darling        Commerce Comets        Right  Right 65  69  70  64  80     65  68     69     63     79        65     69     70     64     80        66     70 61 74 43 0  0  0  45 0  0  0  0  5   FB    91   Normal (3/4) Normal  71  52  41     22     34     76 2 0  0 0 0.0  0   0  0.0 0       0.0   

                                                                                                                                                                                                                                                                                                       BS%_mean  \
POS Name 

In [33]:
print(df_batters.iloc[:, 53:])

      G    GS  PA  AB  H  1B_other  2B_other  3B_other  HR  RBI  R  BB   BB%  \
0     4     4  16  14  4         3         0         0   1    1  3   2  12.5   
1     4     4  18  15  1         1         0         0   0    0  1   3  16.7   
2     3     3  11  10  1         1         0         0   0    0  1   1   9.1   
3     4     4  15  15  3         3         0         0   0    1  0   0   0.0   
4     4     4  14  13  4         4         0         0   0    1  1   1   7.1   
...  ..   ...  ..  .. ..       ...       ...       ...  ..  ... ..  ..   ...   
1127  3  <NA>   4   4  1         1         0         0   0    2  0   0   0.0   
1128  2  <NA>   4   4  0         0         0         0   0    0  0   0   0.0   
1129  2     2   9   9  2         1         1         0   0    1  0   0   0.0   
1130  6     5  19  16  5         5         0         0   0    2  1   3  15.8   
1131  2     2  12   9  1         0         0         0   1    2  1   3  25.0   

      IBB  HP  SH  SF  CI  SO   SO%  GI

In [28]:
df_pitchers_copy

# Analysis